# Filtreler

<img src="https://drive.google.com/uc?id=1itU0IXGCdHis2uY8saTjnOYT51r1tolF" width="400">

[resim kaynağı](https://docs.gimp.org/en/images/filters/examples/convolution-calculate.png)

In [ ]:
from PIL import Image
from matplotlib.pyplot import imshow
import numpy as np
import requests
%matplotlib inline

In [ ]:
ROOT_DIR = "https://raw.githubusercontent.com/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART2/Day13-ComputerVision/notebooks/"

IMAGE_PATH_NOISY_TEST = ROOT_DIR + "images/noisy_test.jpg"
IMAGE_PATH_JET = ROOT_DIR + "images/jet.jpg"
IMAGE_PATH_LENA = ROOT_DIR + "images/lena.jpg"

In [ ]:
#img_noisy = Image.open(IMAGE_PATH_NOISY_TEST)
img_noisy = Image.open(requests.get(IMAGE_PATH_NOISY_TEST, stream=True).raw)

imshow(img_noisy)

## Median(Ortanca Değer) Filtresi

<img src="https://drive.google.com/uc?id=1ymHFml6n18ZqXia1z2QHV9hkNbZpjj8B" width="400">

### Tuz-buz gürültüsünden temizleme
Bir pencere boyutu belirlenir ve resim üzerinde gezdirilen pencere içerisindeki piksel değerleri sıralanarak ortadaki değer alınır.

In [ ]:
img_noisy_array = np.array(img_noisy)
new_img_array = np.array(img_noisy)

## 3x3 Filtre

In [ ]:
def getMedian(li):
    li.sort()
    return li[int(len(li)/2)]

In [ ]:
############### 3x3 window ###############
for i in range(1, img_noisy_array.shape[0] - 1):
    for j in range(1, img_noisy_array.shape[1] - 1):
        win = []
        for x in range(i - 1, i + 2):
            for y in range(j - 1, j + 2):
                win.append(img_noisy_array[x][y])

        new_img_array[i][j] = getMedian(win)

imshow(new_img_array)

## 5x5 Filtre

In [ ]:
############### 5x5 window ###############
img_noisy_array = np.array(img_noisy)
new_img_array = np.array(img_noisy)

for i in range(1, img_noisy_array.shape[0] - 2):
    for j in range(1, img_noisy_array.shape[1] - 2):
        win = []
        for x in range(i - 2, i + 3):
            for y in range(j - 2, j + 3):
                win.append(img_noisy_array[x][y])
        #sort the values
        win.sort()
        
        new_img_array[i][j] = getMedian(win)

imshow(new_img_array)

## Mean (Ortalama) Filtresi
Resim üzerinde gezdirilen pencere içerisindeki piksel değerlerinin ortalaması alınır.

In [ ]:
############### 5x5 window ###############
img_noisy_array = np.array(img_noisy)
new_img_array = np.array(img_noisy)


for i in range(1, img_noisy_array.shape[0] - 2):
    for j in range(1, img_noisy_array.shape[1] - 2):
        win = []
        for x in range(i - 2, i + 3):
            for y in range(j - 2, j + 3):
                win.append(img_noisy_array[x][y])
                
        new_img_array[i][j] = (np.sum(win))/(5*5)

imshow(new_img_array)

<hr>

## Kenar nedir

Piksel yoğunluklarında aşırı farklılıklar bulunan alan genellikle bir nesnenin kenarını gösterir.

### Kenar Çeşitleri

Bunlar, görüntülerde bulunan ana ideal kenar tipleridir. Bir görüntüdeki görüntü yoğunluğunun değişme şekli, o konumda mevcut olan kenar tipini belirler. Şekil 1, piksellerin x ya da y doğrultusundaki mesafelerine karşı olan görüntü yoğunluğunu temsil etmek için bir çizgi kullanarak bu farklılıkları göstermeye yardımcı olur.

- a. Adım Kenarı (Step Edge) - Görüntünün yoğunluğunun derhal bir mesafeden arttığı yerde.
- b. Rampa Kenarı (Ramp Edge) - Görüntünün yoğunluğunun belli bir mesafeye kademeli olarak arttığı yer.
- c. Tavan Kenarı (Roof Edge) - Görüntünün yoğunluğunun geçici olarak belirli bir mesafeye yayıldığı yer.

<img src="https://drive.google.com/uc?id=1e4RMoZgUafg8TFUXYW5CQUwAJKFQO5Pf" width="400">

[Makale ve Resim Referansı](http://www.doc.ic.ac.uk/~ts2615/contribution.html#1)

## Kenar Tespiti: Sobel Filtresi

In [ ]:
img = Image.open(requests.get(IMAGE_PATH_JET, stream=True).raw)
imshow(img)

Sobel, bir görüntünün kenarlarını algılamak için çok yaygın bir operatördür, bu bir görüntünün türevine bir yaklaşımdır. Y ve x yönlerinde ayrıdır. Burada, her x ve y yönü için bir tane olmak üzere bir çekirdek 3 * 3 matrisi kullanıyoruz. X yönü için degrade, solda eksi sayılar, sağında da pozitif sayılar ve merkez pikselleri koruruz. Satır pikselleri.

<img src="https://drive.google.com/uc?id=17XqpNzhDaIPVgUx_haykHKMiFJih4dPu" width="400">


In [ ]:
img_array = np.array(img)

#define horizontal and Vertical sobel kernels
Gx = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
Gy = np.array([[-1, -2, -1],[0, 0, 0],[1, 2, 1]])

### Çekirdek evrişim fonksiyonunu tanımlamak

Evrişim işlevi, aynı boyutta üçüncü bir sayı dizisini üretmek için genellikle farklı boyutlardaki iki sayı dizisini, burada gri skala görüntüsünü ve sobel / prewitt çekirdeklerini bir araya getirme yöntemidir. Evrişim, uygulanan çekirdeğin görüntünün tüm piksellerinin üzerine kaydırılmasıyla gerçekleştirilir.
A, kaynak görüntüdür ve * evrişim işlemini gösterir.

<img src="https://drive.google.com/uc?id=1gc8OLyzVVQ_PXo_Uw-rAZoR-shAMXXFI" width="400">

<img src="https://drive.google.com/uc?id=17rNPM7CpHNBuTh2TS2gA_SwLIYikuGyN" width="400">


In [ ]:
#define kernal convolution function
# with image X and filter F
def convolve(X, F):
    # height and width of the image
    X_height = X.shape[0]
    X_width = X.shape[1]
    
    # height and width of the filter
    F_height = F.shape[0]
    F_width = F.shape[1]
    
    H = (F_height - 1) // 2
    W = (F_width - 1) // 2
    
    #output numpy matrix with height and width
    out = np.zeros((X_height, X_width))
    #iterate over all the pixel of image X
    for i in np.arange(H, X_height-H):
        for j in np.arange(W, X_width-W):
            sum = 0
            #iterate over the filter
            for k in np.arange(-H, H+1):
                for l in np.arange(-W, W+1):
                    #get the corresponding value from image and filter
                    a = X[i+k, j+l]
                    w = F[H+k, W+l]
                    sum += (w * a)
            out[i,j] = sum
    #return convolution  
    return out

### Gradyan büyüklüğü

Her yöndeki gradyan bileşeni daha sonra her noktada gradyanın mutlak büyüklüğünü ve bu gradyanın yönünü bulmak için bir araya getirilir.

In [ ]:
#normalizing the vectors
sob_x = convolve(img_array, Gx) / 9.0
sob_y = convolve(img_array, Gy) / 9.0


#calculate the gradient magnitude of vectors
sob_out = np.sqrt(np.power(sob_x, 2) + np.power(sob_y, 2))
# mapping values from 0 to 255
sob_out = (sob_out / np.max(sob_out)) * 255


imshow(sob_out)